# Data Preprocessing Tools

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

> Pandas Libray를 사용하면 쉽게 데이터셋 파일을 로드가 가능하다.
- CSV File: read_csv('path)
- JSON File: read_json('path')

> 추가로 내장 모듈인 os, json을 이용해 로드도 가능하다.
```python
import os
import json

# JSON 파일 읽기
with open('data.json', 'r') as file:
    data = json.load(file)

# 데이터 출력 (dict 또는 list 형태)
print(data)

# Pandas DataFrame으로 변환
import pandas as pd
df = pd.DataFrame(data)
print(df)

file_path = 'data.csv'

# 파일 존재 여부 확인 후 로드
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(df)
else:
    print("파일이 존재하지 않습니다.")
```

In [3]:
dataset = pd.read_csv('../Data.csv')
dataset.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


`특성 변수(Feature Variable)`
: 모델의 입력으로 사용되는 독립 변수로, 데이터셋에서 예측이나 분석의 기반이 되는 변수

`종속 변수(Target Variable)`
: 모델의 출력 또는 예측하려는 목표 변수로, 특성 변수에 의해 영향을 받는 값

In [19]:
# 특성변수 & 종속 변수
x = dataset.iloc[:,:-1].values
y = dataset.iloc[:,-1].values

In [20]:
print(f"x({len(x)}): {x}")
print(f"y({len(y)}): {y}")

x(10): [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]
y(10): ['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes']


## Taking care of missing data

`결측 데이터 처리`
- 방법1: 결측 데이터 무시 및 삭제
- 방법2: 데이터가 없는 부분을 해당 열의 모든 값의 평균으로 대체

In [24]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(x[:,1:3])
x[:,1:3] = imputer.transform(x[:,1:3])

In [25]:
print(x)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


## Encoding categorical data

> 머신러닝 모델은 수치형 데이터를 처리하므로, 범주형 데이터를 수치형으로 변환하여 모델에 입력할 수 있도록 하기 위해 사용됩니다.
- 범주형 데이터: 데이터 값이 특정 범주(category) 또는 그룹에 속하는 값을 나타내며, 수치적 연산이 아닌 구분이나 라벨로서 의미를 갖는 데이터
- `원핫 인코딩 (One-Hot Encoding)`: 범주형 데이터를 이진 벡터로 변환하여 각 범주를 고유한 열로 표현하고, 값이 해당 범주에 속하면 1, 아니면 0으로 표시하는 인코딩 방식이다.

### Encoding the Independent Variable

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X= np.array(ct.fit_transform(x))
X

array([[1.0, 0.0, 1.0, 0.0, 0.0, 44.0, 72000.0],
       [0.0, 1.0, 0.0, 0.0, 1.0, 27.0, 48000.0],
       [0.0, 1.0, 0.0, 1.0, 0.0, 30.0, 54000.0],
       [0.0, 1.0, 0.0, 0.0, 1.0, 38.0, 61000.0],
       [0.0, 1.0, 0.0, 1.0, 0.0, 40.0, 63777.77777777778],
       [1.0, 0.0, 1.0, 0.0, 0.0, 35.0, 58000.0],
       [0.0, 1.0, 0.0, 0.0, 1.0, 38.77777777777778, 52000.0],
       [1.0, 0.0, 1.0, 0.0, 0.0, 48.0, 79000.0],
       [0.0, 1.0, 0.0, 1.0, 0.0, 50.0, 83000.0],
       [1.0, 0.0, 1.0, 0.0, 0.0, 37.0, 67000.0]], dtype=object)

### Encoding the Dependent Variable

In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(y)
Y

array([0, 1, 0, 0, 1, 1, 0, 1, 0, 1])

## Splitting the dataset into the Training set and Test set

Feature Scaling을 적용하기 전에 데이터셋을 학슴용과 테스트용으로 분할 하는 이유는 데이터 누수(Data Leakage)를 방지하기 위해서 이다.
- 테스트 데이터 정보 유출 방지: Feature Scaling은 데이터 분포를 기반으로 수행되므로, 전체 데이터에 적용하면 테스트 데이터의 통계정보가 학습 과정에 포함될 위험이 있다. 이는 모델 평가의 공정성을 해칠 수 있다.
- 실제 환경 시뮬레이션: 테스트 데이터는 훈련되지 않은 새로운 데이터로 간주되어야 하므로, Feature Scaling은 학습 데이터의 통계정보로만 수행해야 평가가 가능하다.

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=1)

print(f"X_train({len(X_train)}): {X_train}")
print(f"X_test({len(X_test)}): {X_test}")
print(f"Y_train({len(Y_train)}): {Y_train}")
print(f"Y_test({len(Y_test)}): {Y_test}")

X_train(8): [[0.0 1.0 0.0 0.0 1.0 38.77777777777778 52000.0]
 [0.0 1.0 0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 1.0 0.0 0.0 44.0 72000.0]
 [0.0 1.0 0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 0.0 1.0 27.0 48000.0]
 [1.0 0.0 1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 1.0 0.0 0.0 35.0 58000.0]]
X_test(2): [[0.0 1.0 0.0 1.0 0.0 30.0 54000.0]
 [1.0 0.0 1.0 0.0 0.0 37.0 67000.0]]
Y_train(8): [0 1 0 0 1 1 0 1]
Y_test(2): [0 1]


## Feature Scaling

`Feature Scaling`은 머신러닝 모델의 성능을 향상시키기 위해 데이터의 범위를 조정하는 전처리 과정이다.

- **Standardisation**: 데이터를 평균을 0, 표준 편차를 1로 조정하여 값을 표준 정규 분포로 변환하는 방법

\[
z = \frac{x - \mu}{\sigma}
\]

- **Normalisation**: 데이터를 최소값 0, 최대값 1 사이의 범위로 변환하는 방법

\[
x' = \frac{x - \text{min}(x)}{\text{max}(x) - \text{min}(x)}
\]

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:,5:] = sc.fit_transform(X_train[:,5:])
X_test[:,5:] = sc.transform(X_test[:,5:])

array([[0.0, 1.0, 0.0, 0.0, 1.0, -0.19159184384578545,
        -1.0781259408412425],
       [0.0, 1.0, 0.0, 1.0, 0.0, -0.014117293757057777,
        -0.07013167641635372],
       [1.0, 0.0, 1.0, 0.0, 0.0, 0.566708506533324, 0.633562432710455],
       [0.0, 1.0, 0.0, 0.0, 1.0, -0.30453019390224867,
        -0.30786617274297867],
       [0.0, 1.0, 0.0, 0.0, 1.0, -1.9018011447007988, -1.420463615551582],
       [1.0, 0.0, 1.0, 0.0, 0.0, 1.1475343068237058, 1.232653363453549],
       [0.0, 1.0, 0.0, 1.0, 0.0, 1.4379472069688968, 1.5749910381638885],
       [1.0, 0.0, 1.0, 0.0, 0.0, -0.7401495441200351,
        -0.5646194287757332]], dtype=object)